In [1]:
from cnnScratch import Layer
from cnnScratch import Dense , Convolutional
from cnnScratch import Activation , Reshape
from cnnScratch import mse, mse_prime , binary_cross_entropy, binary_cross_entropy_prime
from cnnScratch import Sigmoid , Tanh

import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils

In [2]:
#limit data on binary 1,0 digits and limit our database 
def process_data(x, y, limit):
    zero_index = np.where(y==0)[0][:limit]
    one_index = np.where(y==0)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices,], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32")/255
    y = np_utils.to_categorical(y)
    print(y.shape)
    y = y.reshape(len(y)//2, 2, 1)

In [3]:
# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = process_data(x_train, y_train, 100)
x_test, y_test = process_data(x_test, y_test, 100)

(200, 1)


TypeError: cannot unpack non-iterable NoneType object

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
# training data: 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 28 * 28, 1)
x_train = x_train.astype('float32')
x_train /= 255

In [6]:
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)
y_train = y_train.reshape(y_train.shape[0], 10, 1)

In [7]:
# same for test data: 10000 samples
x_test = x_test.reshape(x_test.shape[0], 28 * 28, 1)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)
y_test = y_test.reshape(y_test.shape[0], 10, 1)

In [11]:
# neural network
network = [
    Dense(28 * 28, 40),
    Tanh(),
    Dense(40, 10),
    Tanh()
]

epochs = 1000
learning_rate = 0.1

In [12]:
# train
for e in range(epochs):
    error = 0
    # train on 1000 samples, since we're not training on GPU...
    for x, y in zip(x_train[:1000], y_train[:1000]):
        # forward
        output = x
        for layer in network:
            output = layer.forward(output)

        # error
        error += mse(y, output)

        # backward
        grad = mse_prime(y, output)
        for layer in reversed(network):
            grad = layer.backward(grad, learning_rate)

    error /= 1000
    print('%d/%d, error=%f' % (e + 1, epochs, error))

1/1000, error=0.866493
2/1000, error=0.773882
3/1000, error=0.709297
4/1000, error=0.649836
5/1000, error=0.553785
6/1000, error=0.425213
7/1000, error=0.299812
8/1000, error=0.232227
9/1000, error=0.210665
10/1000, error=0.202854
11/1000, error=0.196805
12/1000, error=0.192624
13/1000, error=0.189508
14/1000, error=0.187374
15/1000, error=0.185099
16/1000, error=0.182807
17/1000, error=0.181167
18/1000, error=0.179404
19/1000, error=0.177456
20/1000, error=0.175910
21/1000, error=0.174840
22/1000, error=0.173931
23/1000, error=0.173074
24/1000, error=0.172289
25/1000, error=0.171432
26/1000, error=0.170603
27/1000, error=0.170030
28/1000, error=0.168542
29/1000, error=0.166006
30/1000, error=0.158170
31/1000, error=0.145243
32/1000, error=0.120843
33/1000, error=0.111995
34/1000, error=0.108263
35/1000, error=0.106366
36/1000, error=0.104853
37/1000, error=0.103817
38/1000, error=0.102596
39/1000, error=0.101478
40/1000, error=0.100022
41/1000, error=0.099122
42/1000, error=0.097835
4

In [20]:
# test on 20 samples
pred = []
y_wan = []
for x, y in zip(x_test[:20], y_test[:20]):
    output = x
    for layer in network:
        output = layer.forward(output)
    pred.append(np.argmax(output))
    y_wan.append(np.argmax(y))
    print('pred:', np.argmax(output), '\ttrue:', np.argmax(y))
print(pred , "\t", y_wan)

pred: 7 	true: 7
pred: 3 	true: 2
pred: 1 	true: 1
pred: 0 	true: 0
pred: 4 	true: 4
pred: 1 	true: 1
pred: 4 	true: 4
pred: 9 	true: 9
pred: 2 	true: 5
pred: 9 	true: 9
pred: 0 	true: 0
pred: 6 	true: 6
pred: 9 	true: 9
pred: 0 	true: 0
pred: 1 	true: 1
pred: 4 	true: 5
pred: 9 	true: 9
pred: 7 	true: 7
pred: 3 	true: 3
pred: 4 	true: 4
[7, 3, 1, 0, 4, 1, 4, 9, 2, 9, 0, 6, 9, 0, 1, 4, 9, 7, 3, 4] 	 [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4]


In [21]:
import math

MSE = np.square(np.subtract(y_wan,pred)).mean() 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.7416198487095663


In [ ]:
class Model(layer):
    def train():
        # train
        for e in range(epochs): 
            error = 0
            # train on 1000 samples, since we're not training on GPU...
            for x, y in zip(x_train[:1000], y_train[:1000]):
                # forward
                output = x
                for layer in network:
                    output = layer.forward(output)

                # error
                error += mse(y, output)

                # backward
                grad = mse_prime(y, output)
                for layer in reversed(network):
                    grad = layer.backward(grad, learning_rate)

            error /= 1000
            print('%d/%d, error=%f' % (e + 1, epochs, error))
        def test():
            for x, y in zip(x_test[:20], y_test[:20]):
                output = x
                for layer in network:
                    output = layer.forward(output)
                print('pred:', np.argmax(output), '\ttrue:', np.argmax(y))
        
